<a href=https://keras.io/examples/vision/conv_lstm/>LSTM</a><br>
<a href=https://stanford.edu/%7Eshervine/blog/keras-how-to-generate-data-on-the-fly>Dataset Class</a>

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from data import DataGenerator
import matplotlib.pyplot as plt
import os

2023-08-20 18:59:21.836854: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.571909, nms_thresh=0.3.


## Test Stardist

In [2]:
from stardist.models import StarDist2D
from lamprogen.recipes.dl import stardist_2d_slicewise
import tifffile as tiff

modeldir = f'models'
name = 'lamprogen-stardist-trained'
model = StarDist2D(None, name=name,basedir=modeldir)

# im = tiff.imread('data/images/230413 GLA HEX grad post v2 last well_Plate_1082_well_D12_channel2_1.tif')
# labeled_3d = stardist_2d_slicewise(im, model)

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.571909, nms_thresh=0.3.


In [3]:
import napari

im = tiff.imread('data/inputs/230413 GLA HEX grad post v2 last well_Plate_1082_well_D12_channel2_1.tif')
mask = tiff.imread('data/masks/label_300.tif')

np.max(mask)
# viewer=napari.Viewer()

# viewer.add_image(im, name='intensity')
# viewer.add_labels(mask, name='labels')

3141

## Define Dataset

In [4]:
import tifffile as tiff
image = tiff.imread("data/masks/230413 GLA HEX grad post v2 last well_Plate_1082_well_D12_channel2_1.tif")
image.shape

(70, 256, 256)

In [6]:
def get_ids(path='data'):
    images = []
    masks = []
    for file in os.listdir(path+'/inputs'):
        if 'tif' in file:
            images.append(file)

    for file in os.listdir(path+'/masks'):
        if 'tif' in file:
            masks.append(file)

    return images, masks

images, masks = get_ids()
images.sort()
masks.sort()

X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.1)

In [7]:
print(X_train[3])
print(y_train[3])
print(X_test[0])
print(y_test[0])

input_21.tif
label_21.tif
input_318.tif
label_318.tif


In [8]:
batch_size = 32

params = {'dim': (70,256,256),
          'batch_size': batch_size,
          'n_channels': 1,
          'shuffle': True}


training_generator = DataGenerator(X_train, y_train, **params)
validation_generator = DataGenerator(X_test, y_test, **params)

In [9]:
print(training_generator[0][0].shape)
type(training_generator)

[136  67 306 195 318 107  50 294 239  29 266 175 143 145  86 227  55 219
 206 146   0 197  56 171 127 182 204 261 283 126 258 103]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
(32, 70, 256, 256, 1)


data.DataGenerator

In [9]:
# # Construct a figure on which we will visualize the images.
# fig, axes = plt.subplots(7, 10, figsize=(20, 16))

# # Plot each of the sequential images for one random data example.
# image = training_generator[0][0][0]
# for idx, ax in enumerate(axes.flat):
#     ax.imshow(np.squeeze(image[idx].astype('uint8')), cmap="magma")
#     ax.set_title(f"{idx + 1}")
#     ax.axis("off")

# # Print information and display the figure.
# print(f"Displaying frames for example {0}.")
# plt.show()

## Define Model

In [10]:
#Load Packages
import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, ConvLSTM2D, BatchNormalization

model = Sequential()

model.add(ConvLSTM2D(filters=3, kernel_size=(3, 3),
                   input_shape=(70, 256, 256, 1),
                   padding='same', return_sequences=True, activation='relu', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())

model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True, activation='relu', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())

model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True, activation='relu', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())

model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True, activation='relu', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())

model.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last', kernel_initializer='HeUniform'))

optimizer = keras.optimizers.Adam(lr=0.0005)
model.compile(loss='binary_crossentropy', optimizer=optimizer)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 70, 256, 256, 3)   444       
                                                                 
 batch_normalization (Batch  (None, 70, 256, 256, 3)   12        
 Normalization)                                                  
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 70, 256, 256, 40   62080     
                             )                                   
                                                                 
 batch_normalization_1 (Bat  (None, 70, 256, 256, 40   160       
 chNormalization)            )                                   
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 70, 256, 256, 40   115360    
                             )                          

## Define Traing Loop

In [11]:
# Define some callbacks to improve training.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

# Define modifiable training hyperparameters.
epochs = 20

checkpoint_path = "LSTM Training"
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Fit the model to the training data.
model.fit(
    training_generator,
    batch_size = batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr, cp_callback],
)

[0, 1, 2, 3]
Epoch 1/20


ValueError: Unexpected result of `train_function` (Empty logs). This could be due to issues in input pipeline that resulted in an empty dataset. Otherwise, please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [ ]:
batch = []

im1 = tiff.imread('data/inputs/230413 GLA HEX grad post v2 last well_Plate_1082_well_D12_channel2_1.tif')
im1 = np.expand_dims(im1, axis=-1)
batch.append(im1)

im2 = tiff.imread('data/inputs/230413 GLA HEX grad post v2_Plate_1081_well_D05_channel2_0.tif')
im2 = np.expand_dims(im2, axis=-1)
batch.append(im2)
batch = np.asarray(batch)
print(batch.shape)

output = model.predict(batch)

(2, 70, 256, 256, 1)
1/1 [==============================] - 36s 36s/step


In [ ]:
output

array([[[[[nan],
          [nan],
          [nan],
          ...,
          [nan],
          [nan],
          [nan]],

         [[nan],
          [nan],
          [nan],
          ...,
          [nan],
          [nan],
          [nan]],

         [[nan],
          [nan],
          [nan],
          ...,
          [nan],
          [nan],
          [nan]],

         ...,

         [[nan],
          [nan],
          [nan],
          ...,
          [nan],
          [nan],
          [nan]],

         [[nan],
          [nan],
          [nan],
          ...,
          [nan],
          [nan],
          [nan]],

         [[nan],
          [nan],
          [nan],
          ...,
          [nan],
          [nan],
          [nan]]],


        [[[nan],
          [nan],
          [nan],
          ...,
          [nan],
          [nan],
          [nan]],

         [[nan],
          [nan],
          [nan],
          ...,
          [nan],
          [nan],
          [nan]],

         [[nan],
          [nan

In [ ]:
X = np.empty((70,256,256,1))
X[:,:,:,:1]= im

test = X[None, :]
test.shape

ValueError: could not broadcast input array from shape (70,256,256) into shape (70,256,256,1)

In [ ]:
output = model.predict(test)

In [ ]:
output.shape

prediction = np.squeeze(output)
prediction.shape

In [ ]:
viewer=napari.Viewer()

im = tiff.imread('data/inputs/230413 GLA HEX grad post v2 last well_Plate_1082_well_D12_channel2_1.tif')
viewer.add_image(im, name='intensity')
viewer.add_labels(prediction.astype("uint8"), name='prediction')

In [ ]:
output